## Twitter Producer

**Original Author:** Walker Rowe.<br/>
**Additional Info:** Working with Streaming Twitter Data Using Kafka. https://www.bmc.com/blogs/working-streaming-twitter-data-using-kafka/<br/>
**Last Modified:** By Sabrina Kirrane on the 12.01.2021.  

## Opening a spark session for the accounts

In [2]:
# Import the findspark module 
import findspark

# Initialize via the full spark path
findspark.init("/usr/local/spark/")

In [3]:
# Import the SparkSession module
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Twitter Stream") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.      
sc = spark.sparkContext

catalog = spark.catalog

## Setting up Kafka & API access

In [20]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install tweepy

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install kafka-python

In [22]:
# Import the tweepy and kafka modules 
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from kafka import KafkaProducer

# Setup Twitter access token variables
access_token = "1349835045121503244-D9NGZrufch8UPM3vBLNwyO0N8yUuds"
access_token_secret =  "vWi3XroqHqZUth075MGAvz5ab68Kfxz49r87cTZ8q1iIT"
consumer_key =  "0na3Q8ye5OROuiBFLwYGGLtLa"
consumer_secret =  "UnRQYO9gk0lqvMyHHQOAlBPUVg0QlVigwFdlCokMdXIveNH8FT"

In [23]:
# Helper class
class StdOutListener(StreamListener):
    # Called when raw data is received from connection.
    def on_data(self, data):
        # Publish a message to a topic.
        producer.send("Stream_accounts", data.encode('utf-8'))
        # Print out the twitter data (use with caution as it will cause the notebook to become unresponsive)
        # print (data)
        return True
    # Called when a non-200 status code is returned
    def on_error(self, status):
        # Print the status message
        print (status)

## Importing and processing the accounts

We create a list with the TwitterID of all the people we will follow/pagesin order to get tweets from

In [4]:
# Reading the custom csv into a dataframe
data = spark.read.csv("Data/Follow - Sheet1.csv", header=True)

# Selecting only the TwitterIDs and converting to a list
accounts = list(data.select('ID').toPandas()['ID'])

## Implementing the Stream

In [24]:
# Encapsulate the code in try except blocks
try:    
    # Publish a message to a topic
    producer = KafkaProducer(bootstrap_servers='localhost:9092', api_version=(0,11,5))

    # Create a StdOutListener object
    l = StdOutListener()

    # Create a OAuth authentication handler object
    auth = OAuthHandler(consumer_key, consumer_secret)

    # Set the tweepy access tokenes
    auth.set_access_token(access_token, access_token_secret)
    
    # Create a Stream object
    stream = Stream(auth, l)
    
    # Filter the stream for all tweets containing the search terms. 
    stream = stream.filter(follow = accounts, languages = ['en'])

except:
    #Print the error
    print("Unexpected error:", sys.exc_info()[0])

Unexpected error: <class 'KeyboardInterrupt'>
